<div style="display: inline-block;">
    <img src="images/nhsa_logo.png" alt="Image" style="text-align: left;">
</div>

# Parent Gauge Data Analysis Project
---
## Data Cleaning Script and Documentation

In this script, we will provide a step-by-step demonstration of how script is being cleaned. 

In [4]:
#install necessary libraries for data cleaning.
!pip3 install pandas
!pip3 install numpy
!pip3 install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 1.4 MB/s eta 0:00:0000:0100:01


In [31]:
# Start with the necessary imports
import pandas as pd
import numpy as np

In [32]:
#Load the Data into the dataframe
df = pd.read_excel('INTVDATA.xlsx', sheet_name ='Main', engine ='openpyxl')

#Copy existing dataframe to .csv file
df.to_csv("intv_data.csv", index=False)

#read the new .csv file
df = pd.read_csv('intv_data.csv')

/var/folders/vw/1sthm_ts1dj_1hqg1xnsz19w0000gn/T/ipykernel_83954/2594145135.py:8: DtypeWarning: Columns (16,18,27,35,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('intv_data.csv')


---------

In [33]:
#Because the main dataset is too large for data cleaning, 
#construct a small sample for faster processing
df_sample = df.sample(frac=0.08)
df_sample.to_csv('sample_data.csv', index=False)

## Main Data Cleaning

This is a summary of all the data cleaning and reformatting steps that were conducted.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.

In [53]:
##REMOVE UNNECESSARY COLUMNS
# guardian_vendor_id, interview_id, interviewer_id, interviewer,
#interviewer_vendor_id, student_staff_vendor_id, student_vendor_id

# List of columns to be removed
columns_to_remove = ['guardian_vendor_id', 'interview_id', 'interviewer_id', 
                     'interviewer', 'interviewer_vendor_id', 'student_staff_vendor_id', 
                     'student_vendor_id']

# Removing the columns from the DataFrame
df_sample = df_sample.drop(columns=columns_to_remove)

#save updates to working csv
df_sample.to_csv('sample_data.csv', index=False)

KeyError: "['guardian_vendor_id', 'interview_id', 'interviewer_id', 'interviewer', 'interviewer_vendor_id', 'student_staff_vendor_id', 'student_vendor_id'] not found in axis"

In [34]:
##code to clean
##PROGRAM
unique_programs = df_sample['program'].unique().tolist()

# Sort the list in place
unique_programs.sort()

#create a text file of the unique programs
with open('unique_programs.txt', 'w') as f:
    for item in unique_programs:
        f.write("%s\n" % item)

In [39]:
##DATE##
# Convert 'date' column to datetime format
#errors=coercse converts problematic dates to NaN.
#df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')
#print(df['date'].isnull().sum())

# Make a copy of the 'date' column
#df_copy = df['date'].copy()

# Iterate over the entries in the 'date' column
for i, date in enumerate(df_sample['date']):
    try:
        # Try to convert the date to datetime format
        pd.to_datetime(date, format='mixed')
    except Exception:
        print(f"An error occurred at index {i} with the date: {date}")
        
df_sample['date'] = pd.to_datetime(df_sample['date'], errors='coerce')

#Create separate 'year', 'month', and 'day' columns
df_sample['year'] = df_sample['date'].dt.year
df_sample['month'] = df_sample['date'].dt.month
df_sample['day'] = df_sample['date'].dt.day

#save to csv
df_sample.to_csv('sample_data.csv', index=False)


In [45]:
##Evaluation##
#use one-hot encoding to create dummy variables in preparation for regression.
#NOTE: This, however eliminates the original 'evaluation' column
df_sample = pd.get_dummies(df_sample, columns=['evaluation'])

#save updates to csv
df_sample.to_csv('sample_data.csv', index=False)


Index(['Unnamed: 0.1', 'Unnamed: 0', 'center', 'created_at', 'date',
       'evaluation_year', 'guardian', 'guardian_employment',
       'guardian_enrollment_date', 'guardian_highest_education',
       ...
       'OQ7', 'OQ8', 'OQ9', 'year', 'month', 'day', 'evaluation_Initial',
       'evaluation_Midyear', 'evaluation_Post', 'evaluation_evaluation'],
      dtype='object', length=107)


In [48]:
##evaluation_year##
# Split the 'evaluation_year' column into two separate columns 'start_year' and 'end_year'
df_sample[['evaluation_start_year', 'evaluation_end_year']] = df_sample['evaluation_year'].str.split('-', expand=True)

#save updates to csv
df_sample.to_csv('sample_data.csv', index=False)

In [ ]:
##guardian_birth_date##
#– use DOB and interview year to determine guardian’s age during time of interview. 

In [50]:
##guardian_sex##
# Using direct mapping to create dummy variable out of guardian_sex
df_sample['female'] = (df_sample['guardian_sex'] == 'Female').astype(int)

#save updates to csv
df_sample.to_csv('sample_data.csv', index=False)